## IndicTrans2

In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
# from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer
from IndicTransToolkit import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
from torch.nn.functional import softmax


BATCH_SIZE = 4 # edited from 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
print(DEVICE)

cuda


In [2]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model

**TODO: CAN I USE SOME CACHE TO STORE THE HF model and tokenizer?**

In [3]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir,  quantization)

ip_en_ind = IndicProcessor(inference=True)

In [4]:
with en_indic_tokenizer.as_target_tokenizer():
   print(en_indic_tokenizer.batch_decode([8911]))

['नानी ']


/home/sofia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
def resolve_logits_for_best_beam(outputs, num_beams):
    """ Resolve the logits from the best beam, using model output from a generate call.
        For a shape [tokens?, batch_size*num_beams, vocab], returns [tokens?, batch_size, vocab]

        Assumes num_return_sequences=1."""
    

    best_logits  = []
    beam_indices = [ outputs.beam_indices[:,i].tolist() for i in range(len(outputs.logits)-1) ]

    for beam_index, logits in zip(beam_indices, outputs.logits):
        beam_index = [ idx if idx != -1 else ((num_beams*(i+1))-1) for i, idx in enumerate(beam_index) ]
        best_logits.append(logits[beam_index,:])

    return best_logits

def get_logits_for_span(logits, sequences, tokenizer, search_spans):
    """ Given search spans, returns the logits before the span was generated.

    Args:
        logits (tuple[Tensor]): Tuple of tensors, of shape [tokens?, batch_size, vocab]
        sequences (tuple[list[int]]): Tokenized output sequences.
        tokenizer (PreTrainedTokenizerBase): Tokenizer for the model.
        search_spans (list[str]): batch_size spans to search for. Must be present in the generated sequences.

    Returns:
        Tensor: Tensor of shape [batch_size, vocab] indicating the logits before the span for each batch element.
    """
    # print("sequences", sequences)
    print("lengt of sequences", len(sequences))   

    if isinstance(search_spans, str):
        search_spans = [ search_spans ] * len(sequences)

    # print("search_spans", search_spans) 

    with tokenizer.as_target_tokenizer():   
        detok_outputs = tokenizer.batch_decode(sequences, skip_special_tokens=True)

    # print("detok_outputs", detok_outputs)

    positions = [ output.index(span) for output, span in zip(detok_outputs, search_spans) ]
    # print("positions", positions)
    # print("detok_outputs[positions[0]]", detok_outputs[0][5])
    logit_pos = [  ]

    for seq, detok_seq, span, pos in zip(sequences, detok_outputs, search_spans, positions): #TODO: NOT RUNNING FOR ALL SEQUENCES -- use zip_logest??
        print("seq", seq, "logit_pos", logit_pos)
        if pos == 0:
            with tokenizer.as_target_tokenizer():
                subtokens = tokenizer(span, add_special_tokens=False).input_ids
                # print("subtokens if 1", subtokens)
            # subtokens = tokenizer(span, add_special_tokens=False).input_ids
        else:
            with tokenizer.as_target_tokenizer():
                subtokens = tokenizer(span, add_special_tokens=False).input_ids
                # print("subtokens else 1", subtokens)
                # print("detok_seq[pos-1]", detok_seq[pos-1],"--", detok_seq[pos-1] + span, "--")    
                # subtokens = tokenizer(span, add_special_tokens=False).input_ids
                # subtokens_2 = tokenizer(detok_seq[pos-1] + span, add_special_tokens=False).input_ids # TODO WHY THIS?
                # # print("subtokens_2 else 1", subtokens_2)
                # if subtokens[0] not in subtokens_2: 
                #     subtokens = subtokens_2

        # print("subtokens effective", subtokens)
        # print("seq::", seq)
        idx = 0
        while idx < len(seq):
            if all(seq[idx+i] == tok for i, tok in enumerate(subtokens)): 
                # print( "found", idx)
                break
            idx += 1
        logit_pos.append(idx)
        # print("subtokens", subtokens)
        # print("idx", idx)
        # print("logit_pos", logit_pos)   

    # expand the for loop and create a variable ans_stack for torch.stack([ logits[token][batch,:] for batch, token in enumerate(logit_pos) ]) and return it
    selected_logits = []

    # Iterate over each batch and corresponding token position
    for batch, token in enumerate(logit_pos):
        # print("batch, token", batch, token)
        # print("len(logits)", len(logits))
        # print("Shape of logits", logits[0].shape)
        # print("logits 1st item 1st row",logits[0][0])
        # Extract logits for the specific token position in the current batch
        # current_logit = logits[token][batch, :]
        current_logit = logits[token][0]
        
        # Append the selected logit to the list
        selected_logits.append(current_logit)

    # Stack the list of selected logits into a tensor
    selected_logits = torch.stack(selected_logits)

    return selected_logits
    # return torch.stack([ logits[token][batch,:] for batch, token in enumerate(logit_pos) ])

    # return torch.gather(output.scores, index=, dim=-2)



In [6]:



def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in tqdm(range(0, len(input_sentences), BATCH_SIZE)):
        batch = input_sentences[i : i + BATCH_SIZE]
        # print("batch: ", batch)

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        # print("batch after pre-process: ", batch)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)
        print("inputs: ", inputs)

        # Generate translations using the model
        with torch.no_grad():
            # generated_tokens = model.generate(
            outputs = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
                output_scores=True,
                output_logits=True,
                return_dict_in_generate=True,

            )
            
            outputs.beam_indices = outputs.beam_indices.cpu()
            # print("beam_indices: ",len(outputs.beam_indices[0]), outputs.beam_indices)
            outputs.logits = tuple(logits.cpu() for logits in outputs.logits)               
            # print("Beam index logits: ", outputs.logits)
        # Decode the generated tokens into text
        generated_tokens = outputs.sequences
        # print("len generated_tokens: ", (generated_tokens[0]).shape)
        vector = generated_tokens.detach().cpu().tolist()
        print("length of outputs vectors: ", len(vector), len(vector[0]))
        # print("vector of generated_tokens: ", vector)



        with tokenizer.as_target_tokenizer():
            decoded_op = tokenizer.batch_decode(
                vector,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )
        print(" length decoded_op: ", len(decoded_op), len(decoded_op[0]))
         
        # 1. Resolve logits for the best beam from these inputs
        #    (would be a tuple of tensoors of shape of [tokens?, batch_size, vocab])
        best_logits = resolve_logits_for_best_beam(outputs, num_beams=5)
        # print("best_logits: ", best_logits)

        # 2. Get the logits before New Delhi
        #    (would be a tensor of shape of [batch_size, vocab])
        start_logits = get_logits_for_span(best_logits, outputs.sequences, tokenizer, [ "नानी"]*BATCH_SIZE)
        print( "start_logits: ", start_logits)
        print("Length of start_logits: ", len(start_logits), len(start_logits[0]))
        # TODO convert to prob by using softmax


        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(decoded_op, lang=tgt_lang)



        del inputs
        torch.cuda.empty_cache()

    return translations

In [7]:
# import the test_sentences_eng.txt file data as sents

# sents = ['The river is blue.', 'My uncle wearing blue coat.', 'My maternal aunt went to the river bank.', "The doctor went to bank for money."]
sents=['My maternal aunt went to the river bank.', 'My maternal aunt loves me.', 
       'My maternal aunt loves her grandchildren.', "My maternal aunt loves her children.", 
       "My maternal aunt loves her family.",
       ]   
src_lang = "eng_Latn"

tgt_lang = 'hin_Deva'
# print(lang)
translations = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


  0%|          | 0/2 [00:00<?, ?it/s]

inputs:  {'input_ids': tensor([[    4,    15,   362, 14624, 17973,   436,     9,     6,  1811,   978,
             5,     2],
        [    1,     1,     1,     4,    15,   362, 14624, 17973,  3197,   102,
             5,     2],
        [    1,     1,     4,    15,   362, 14624, 17973,  3197,   107, 15964,
             5,     2],
        [    1,     1,     4,    15,   362, 14624, 17973,  3197,   107,   211,
             5,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


 50%|█████     | 1/2 [00:00<00:00,  1.13it/s]

length of outputs vectors:  4 14
 length decoded_op:  4 33
lengt of sequences 4
seq tensor([    2,  1653,  8911,  3228,     9, 15933,   167,  3070,     6,     2,
            1,     1,     1,     1], device='cuda:0') logit_pos []
seq tensor([    2,  1653,  8911, 13343,  2507,  1423,    33,     6,     2,     1,
            1,     1,     1,     1], device='cuda:0') logit_pos [2]
seq tensor([    2,  1653,  8911,   219, 10466,    13, 14855,   458,    20,  2507,
         1423,     8,     6,     2], device='cuda:0') logit_pos [2, 2]
seq tensor([   2, 1653, 8911,  219, 2208,   20, 2507, 1423,   33,    6,    2,    1,
           1,    1], device='cuda:0') logit_pos [2, 2, 2]
start_logits:  tensor([[-0.9370, -0.9375,  0.2827,  ..., -0.9375, -0.9375, -0.9375],
        [-0.9370, -0.9375,  0.2827,  ..., -0.9375, -0.9375, -0.9375],
        [-0.9370, -0.9375,  0.2827,  ..., -0.9375, -0.9375, -0.9375],
        [-0.9370, -0.9375,  0.2827,  ..., -0.9375, -0.9375, -0.9375]],
       dtype=torch.float16)
Le

100%|██████████| 2/2 [00:01<00:00,  1.62it/s]

length of outputs vectors:  1 11
 length decoded_op:  1 42
lengt of sequences 1
seq tensor([   2, 1653, 8911,  219,  947,   20, 2507, 1423,   33,    6,    2],
       device='cuda:0') logit_pos []
start_logits:  tensor([[-0.9678, -0.9678,  1.3027,  ..., -0.9678, -0.9678, -0.9678]],
       dtype=torch.float16)
Length of start_logits:  1 122672


In [8]:
print(translations)

['मेरी नानी नदी के किनारे गई थीं। ', 'मेरी नानी मुझसे प्यार करती हैं। ', 'मेरी नानी अपने पोते-पोतियों से प्यार करती है। ', 'मेरी नानी अपने बच्चों से प्यार करती हैं। ', 'मेरी नानी अपने परिवार से प्यार करती हैं। ']
